# Lessons 1 & 2: Large Language Models — Condensed Practical Notebook (EN)
## Quick intro to LLMs, basic Transformer ideas, and hands-on usage with Groq and Google AI Studio

Course: Building AI Agentic Systems for Advertising Campaign Analysis with LangChain

Version: 2.0 (October 2025)

---

## Contents (Condensed)

This notebook focuses on doing, not theory. For full theory and slides, see:
- Lesson 1 (EN): ../lesson_01/Lesson 1 EN.md
- Lesson 2 (EN): ../lesson_02/Lesson 2 EN.md

What you’ll do here:
1. Minimal LLM background (one-page summary)
2. API setup for Groq and Gemini (Google AI Studio)
3. Hello LLM: first calls
4. Generation parameters: temperature and max tokens
5. Prompt patterns: instruction, few-shot, CoT
6. Hallucination awareness and mitigation basics
7. Output validation patterns
8. Mini-exercises (8 tasks) with solutions hinted

Estimated time: 2–3 hours

---

# Part 1 — What is an LLM (very short)

Large Language Models (LLMs) are neural networks trained on diverse text to predict the next token. They can summarize, translate, answer questions, write code, and more. Modern LLMs are built on the Transformer architecture (self-attention) and are often fine-tuned or instructed to follow user prompts.

Key ideas to remember:
- Context window: how many tokens the model can consider at once.
- Temperature: higher → more creative/variable; lower → more deterministic.
- System + user prompts: structure and clarity matter more than fancy wording.
- Validation: treat model outputs as drafts; verify facts and numbers.

For deeper theory: see Lesson 1 EN.md and Lesson 2 EN.md.

---

# Part 2 — API Setup (Groq + Gemini)

In this notebook we’ll use these providers and models:
- Groq (OpenAI-compatible API): model `gpt-oss-20b`
  - Console: https://console.groq.com
  - Docs: https://groq.com
- Google AI Studio (Gemini): model `gemini-2.5-flash`
  - Console/API keys: https://aistudio.google.com
  - Python SDK: `google-generativeai`

Notes on keys:
- Put your keys into `GROQ_API_KEY` and `GEMINI_API_KEY` (either inline or as environment variables). If not set, calls will simply fail with a clear error message.


In [ ]:
# Install the minimal libraries used in this notebook
# You can skip this cell if your environment already has them.
# !pip install openai google-generativeai python-dotenv


: 

In [6]:
!pip install openai python-dotenv google-generativeai

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
  ERROR: Could not find a version that satisfies the requirement google-generativeai (from versions: none)
ERROR: No matching distribution found for google-generativeai
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import os
from typing import Any

# Groq uses the OpenAI-compatible SDK
from openai import OpenAI
import google.generativeai as genai

# Option A (not recommended for shared code): paste keys directly
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")

if not GROQ_API_KEY:
    print("Note: GROQ_API_KEY is not set. Set it to call Groq.")
if not GEMINI_API_KEY:
    print("Note: GEMINI_API_KEY is not set. Set it to call Gemini.")

# Configure clients if keys exist (won't crash if missing; we'll handle errors on call)
_groq_client: Any = None
_gemini_ready: bool = False

if GROQ_API_KEY:
    _groq_client = OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")

if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
    _gemini_ready = True

print("Clients prepared (if keys are set).")


ModuleNotFoundError: No module named 'openai'

### Helper functions

- `call_groq` — uses Groq’s OpenAI-compatible endpoint (model: `gpt-oss-20b`)
- `call_gemini` — uses Google’s Gemini SDK (model: `gemini-2.5-flash`)
- `call_llm` — unified interface: `provider` in {`groq`, `gemini`}


In [ ]:
from typing import Optional


def call_groq(prompt: str,
              temperature: float = 0.0,
              max_tokens: int = 800,
              model: str = "gpt-oss-20b") -> str:
    """Call Groq via OpenAI-compatible API.

    Args:
        prompt: User prompt.
        temperature: 0.0–2.0. Lower = more deterministic.
        max_tokens: Max tokens in the response.
        model: Groq model name.
    Returns:
        Model response text or an error message.
    """
    if _groq_client is None:
        return "Groq client not configured. Set GROQ_API_KEY and re-run."
    try:
        response = _groq_client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Groq call failed: {e}"


def call_gemini(prompt: str,
                temperature: float = 0.0,
                max_tokens: int = 800,
                model: str = "gemini-2.5-flash") -> str:
    """Call Google Gemini via google-generativeai SDK.

    Args:
        prompt: User prompt.
        temperature: 0.0–2.0.
        max_tokens: Max output tokens.
        model: Gemini model name.
    Returns:
        Model response text or an error message.
    """
    if not _gemini_ready:
        return "Gemini not configured. Set GEMINI_API_KEY and re-run."
    try:
        model_instance = genai.GenerativeModel(model)
        generation_config = genai.types.GenerationConfig(
            temperature=temperature,
            max_output_tokens=max_tokens,
        )
        response = model_instance.generate_content(prompt, generation_config=generation_config)
        return (response.text or "").strip()
    except Exception as e:
        return f"Gemini call failed: {e}"


def call_llm(prompt: str,
             provider: str = "groq",
             temperature: float = 0.0,
             max_tokens: int = 800,
             model: Optional[str] = None,
             verbose: bool = True) -> str:
    """Unified interface to call Groq or Gemini."""
    provider_norm = provider.lower().strip()

    if provider_norm == "groq":
        model = model or "gpt-oss-20b"
        if verbose:
            print(f"[Groq] model={model} temperature={temperature} max_tokens={max_tokens}")
        return call_groq(prompt, temperature=temperature, max_tokens=max_tokens, model=model)

    if provider_norm == "gemini":
        model = model or "gemini-2.5-flash"
        if verbose:
            print(f"[Gemini] model={model} temperature={temperature} max_tokens={max_tokens}")
        return call_gemini(prompt, temperature=temperature, max_tokens=max_tokens, model=model)

    return "Unknown provider. Use 'groq' or 'gemini'."


# Part 3 — First calls

Run a simple prompt on both providers to verify your setup.


In [ ]:
prompt = "Summarize in one sentence why validation is important when using LLMs in analytics."

print("== Groq ==")
print(call_llm(prompt, provider="groq", verbose=False))

print("\n== Gemini ==")
print(call_llm(prompt, provider="gemini", verbose=False))


# Part 4 — Generation parameters

Two important knobs:
- Temperature controls randomness.
- Max tokens caps the length of the answer.

Try different temperatures and compare.


In [ ]:
question = "Give three diverse brainstorming ideas for a TV ad about eco-friendly travel, one line each."
for temp in [0.0, 0.3, 0.7, 0.9]:
    print(f"\n--- Temperature={temp} (Groq) ---")
    print(call_llm(question, provider="groq", temperature=temp, max_tokens=150, verbose=False))


In [ ]:
"""You are an expert {role}. You produce accurate, structured and reliable outputs.
Your task is to {task}.
Focus on: {priorità}.

CONTEXT:
- {contesto 1}
- {contesto 2}
- {contesto 3}

RULES:
- Style: {stile}
- Format: {formato}
- Length: {lunghezza}
- Avoid: {cose da evitare}

EXAMPLE (optional):
{eventuale esempio}

OUTPUT:
Respond using the following structure:
{struttura di output desiderata}"""

SyntaxError: unterminated string literal (detected at line 1) (2816687793.py, line 1)

# Part 5 — Prompt patterns

We’ll practice three common patterns: instruction, few-shot, and chain-of-thought (CoT).

Note: Avoid revealing CoT in production outputs. Use CoT internally or request explanations only when needed.


### 5.1 Instruction prompt

Keep it short, specify format, and define constraints.


In [ ]:
inst = (
    "You are a helpful marketing analyst.\n"
    "Task: Write a 50-word summary about the target audience A25-54 for a sports streaming service.\n"
    "Constraints: Use plain English; include 1 key behavior and 1 pain point."
)
print(call_llm(inst, provider="groq", verbose=False))


### 5.2 Few-shot prompt

Provide 1–3 examples to nudge the model toward your preferred style.


In [ ]:
few_shot = (
    """Dammi la durata potenziale di questa batteria e dimmi per quale finalità la usi. Analizzami il documento dell'utilizzo della batteria e calcolami la durata come chiesto.
    Output:
    - Example 1: '{"tempo": 3h, "task": luce accesa}'
    - Example 1: '{"tempo": 2h, "task":lavatrice in standby}'
    - Example 1: '{"tempo": 10h, "task": led acceso}'"""
)
print(call_llm(few_shot, provider="gemini", verbose=False))


NameError: name 'call_llm' is not defined

### 5.3 Chain-of-Thought (CoT) vs. direct answer

First, ask for an answer without reasoning; then ask for a step-by-step explanation.


In [ ]:
no_cot = (
    "Classify the statement as positive, neutral, or negative: 'The campaign reached 45% but frequency was only 1.1.'\n"
    "Answer with one word."
)
with_cot = (
    "Classify the statement as positive, neutral, or negative: 'The campaign reached 45% but frequency was only 1.1.'\n"
    "Explain your reasoning briefly, then give the final label."
)
print("No-CoT:\n", call_llm(no_cot, provider="groq", verbose=False))
print("\nWith CoT:\n", call_llm(with_cot, provider="groq", verbose=False))


# Part 6 — Hallucination awareness (lightweight)

LLMs can produce confident but incorrect answers. Tactics:
- Ask for sources or confidence ranges (still not guarantees).
- Provide context within the prompt (grounding).
- Keep tasks constrained and verifiable.

Try a grounded prompt by giving the model the relevant facts first.


In [ ]:
context = (
    "Facts:\n- Budget: €500k\n- Channels: TV + CTV\n- Goal: Increase A25-54 reach to 55% with frequency ≥ 3\n\n"
    "Question: Propose two brief tactics that align with the goal."
)
print(call_llm(context, provider="gemini", verbose=False))


# Part 7 — Output validation patterns

Examples:
- Ask the model to output JSON that you can parse and validate.
- Add simple schema rules in the prompt.
- Post-validate in Python; if it fails, ask the model to correct.


In [ ]:
validation_prompt = (
    "Return a JSON object with keys 'channel', 'reach_target', 'confidence' and 'risk_note'.\n"
    "Constraints: 'reach_target' must be an integer (40–80). Use double quotes."
)
resp = call_llm(validation_prompt, provider="groq", temperature=0.2, verbose=False)
print(resp)


# Part 8 — Mini-exercises (8)

Instructions: try on both providers (Groq and Gemini) when possible.

1) Temperature sweep: generate 5 headline variants for a sustainability campaign; compare 0.0 vs 1.0.
2) Format control: ask for a 3-row Markdown table with columns: Metric | Definition | Why it matters.
3) Style mimic: provide one example claim and ask the model to mimic the style for a new product.
4) Guarded claims: give budget and reach goals; ask the model to propose 3 tactics and include 1 risk per tactic.
5) CoT: ask for a step-by-step short reasoning to classify performance as good/ok/poor.
6) JSON output: request a schema and validate keys in Python; if invalid, ask the model to fix.
7) Prompt repair: give a poor prompt; ask the model to rewrite it to be precise and testable.
8) Provider compare: run the same prompt on Groq and Gemini and note differences in style/latency.

Solutions: see comments and hints in each previous section; adapt as needed.


# Appendix — Useful links

- Groq: https://groq.com — Console: https://console.groq.com
- Google AI Studio (Gemini): https://aistudio.google.com — Python SDK: https://github.com/google-gemini/generative-ai-python
- Prompting basics: https://platform.openai.com/docs/guides/prompt-engineering (general concepts)

End of condensed notebook.
